In [407]:
import urllib3
import os
import requests
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import time
from lxml import etree
import re
import pathlib
import flask
import jinja2

In [408]:
requests.adapters.DEFAULT_RETRIES =5
# os.environ['CURL_CA_BUNDLE'] = ''
doc_path = r'C:\Users\Elysia\Desktop\learn\课件\二下\python\lab_1\html_doc_'

In [409]:
lsdir = os.listdir('.')
if 'html_doc' not in lsdir:
    os.mkdir('./html_doc_')
if 'dataset' not in lsdir:
    os.mkdir('./dataset')
os.chmod('./html_doc_',0o777)
aim = 'hug_dataset'
html_path = os.path.join(os.getcwd(),'html_doc_')
web_path = os.path.join(os.getcwd(),'dataset_')
viewer_path = os.path.join(os.getcwd(),'viewer_')

In [410]:
HTML_DOC = dict()
dir = os.listdir(doc_path)
for k in dir:
    with open(os.path.join(doc_path,k),'r',encoding='utf-8') as f:
        HTML_DOC[k] = f.read()
# for k,v in HTML_DOC.items():
#     print(k,v)

In [411]:
def find_page_link(html_doc):
    """
    页面链接
    """
    soup = bs(html_doc,'lxml')
    label = soup.find('h1',attrs={
        'class':"flex flex-wrap items-center leading-tight mb-3 text-lg md:text-xl"
                                       })
    inner = label.contents
    page_link = ''
    err = '!!!'
    # try:
    #     page_link += inner[0].span.string + ' '
    # except Exception:
    #     page_link += err
    try:
        page_link += inner[2].a.string + ' '
    except Exception:
        page_link += err
    try:
        page_link += inner[4].a.string + ' '
    except Exception:
        page_link += err
    try:
        page_link += inner[6].find_all('button')[1].string + ' '
    except Exception:
        page_link += err
    return page_link

def find_download_time(html_doc):
    soup = bs(html_doc,'lxml')
    # print(soup.html.body.div.main.find_all('div')[2])

    dom = etree.HTML(html_doc)
    download_time = -1
    try:
        download_time = dom.xpath('/html/body/div/main/div[2]/section[2]/dl/dd/text()')[0]
        
    except Exception:
        pass
    return download_time



# def find_dataset_numb(html_doc):
#     soup = bs(html_doc,'lxml')
#     dom = etree.HTML(html_doc)
#     dataset_numb = -1
#     try:
#         dataset_numb = dom.xpath('/html/body/div/main/div[2]/section[2]/div[4]/a[6]/div[2]/text()')[1]
#     except Exception:
#         pass
#     # print(dataset_numb)
#     return dataset_numb

def find_dataset_numb(html_doc):
    index_1 = html_doc.rfind('Number of rows')
    # print(index_1)
    # print(html_doc[index_1:index_1+100])
    index_2 = html_doc[index_1:].find('<!-- HTML_TAG_START -->') + len('<!-- HTML_TAG_START -->')
    # print(html_doc[index_1:][index_2:index_2+100])
    index_3 = html_doc[index_1:].find('<!-- HTML_TAG_END -->')
    ret = html_doc[index_1:][index_2:index_3]
    if len(ret) == 0:
        ret = '-1'
    return ret


def find_dataset_size(html_doc:str):
    index_1 = html_doc.rfind('Size of the auto-converted Parquet files')
    index_2 = html_doc[index_1:].find('<!-- HTML_TAG_START -->') + len('<!-- HTML_TAG_START -->')
    index_3 = html_doc[index_1:].find('<!-- HTML_TAG_END -->')
    re = html_doc[index_1:][index_2:index_3]
    if 'B' in re:
        re_num = eval(re[:re.find('B')-1])
        if 'K' in re:
            re_num = re_num / 1024
        elif 'M' in re:
            re_num = re_num
        elif 'G' in re:
            re_num = re_num * 1024
        elif 'T' in re:
            re_num = re_num * 1024 * 1024
        else:
            re_num = re_num / (1024 * 1024)
        return re_num
    else:
        return -1
p = 0
def find_dataset_viewer(file_name,html_doc):
    global p
    soup = bs(html_doc,'lxml')
    dom = etree.HTML(html_doc)
    try:
        try:
            # dataset_numb = soup.html.body.div.main.find_all('div')[2]
            dataset_numb = dom.xpath('/html/body/div/main/div[2]/section[1]/div[5]/div')[0]
            # text = etree.tostring(dataset_numb)
            # soup_ = bs(text,'lxml').html.body
            # p = soup_
            # writeinner = soup_.get_text()
            # text = re.sub(r'[\r\n]+', '\n', writeinner)
            # with open(os.path.join(viewer_path,file_name+'.html'),'bw') as f:
            #     # f.write('method_1')
            #     f.write(text.encode('utf-8'))
            # print(1)

                
        except Exception:
            # print(2)
            dataset_numb = dom.xpath('/html/body/div/main/div[2]/section[1]/div[4]')[0]
        
        
        text = etree.tostring(dataset_numb)
        soup_ = bs(text,'lxml').html.body
        p = soup_
        writeinner = soup_.get_text()
        text = re.sub(r'[\r\n]+', '\n', writeinner)
        with open(os.path.join(viewer_path,file_name+'.html'),'bw') as f:
            # f.write('method_1')
            f.write(text.encode('utf-8'))
        
            
    except Exception:
        pass
    #     with open(os.path.join(viewer_path,file_name+'.html'),'wb') as f:
    #     # for data in dataset_numb:
    #         f.write('ERROR'.encode('utf-8'))
    return(os.path.join(viewer_path,file_name+'.html'))

In [413]:

print('--------------------------------')
for k,v in HTML_DOC.items():
    # print('k',k)
    page_link = find_page_link(v)
    download_time = find_download_time(v)
    dataset_size = find_dataset_size(v)
    dataset_numb = find_dataset_numb(v)
    viewer_paths = find_dataset_viewer(k,v)
    print('page_link:',page_link)
    print('download_time:',download_time)
    print('dataset_size:',dataset_size,'MB')
    print('dataset_numb:',dataset_numb)
    print('viewer_path',viewer_paths)
    print('--------------------------------')

--------------------------------
page_link: DIBT 10k_prompts_ranked 84 
download_time: 1,219
dataset_size: 3.58 MB
dataset_numb: 10,331
viewer_path c:\Users\Elysia\Desktop\learn\课件\二下\python\lab_1\viewer_\DIBT_10k_prompts_ranked · Datasets at Hugging Face.html.html
--------------------------------
page_link: liuhaotian LLaVA-Instruct-150K 299 
download_time: 203
dataset_size: -1 MB
dataset_numb: -1
viewer_path c:\Users\Elysia\Desktop\learn\课件\二下\python\lab_1\viewer_\liuhaotian_LLaVA-Instruct-150K · Datasets at Hugging Face.html.html
--------------------------------
page_link: Locutusque UltraTextbooks-2.0 20 
download_time: 183
dataset_size: 6287.36 MB
dataset_numb: 3,220,278
viewer_path c:\Users\Elysia\Desktop\learn\课件\二下\python\lab_1\viewer_\Locutusque_UltraTextbooks-2.0 · Datasets at Hugging Face.html.html
--------------------------------
page_link: maharshipandya spotify-tracks-dataset 46 
download_time: 1,082
dataset_size: 13.6 MB
dataset_numb: 114,000
viewer_path c:\Users\Elysia\